In [1]:
!pip install imblearn

In [49]:
import torch
import torch.nn as nn
from imblearn.over_sampling import SMOTE
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from torchvision import datasets, transforms, utils
from torch.utils import data

In [50]:
df=pd.read_excel('./modeling_data/Result2.xlsx')

In [51]:
X=df.iloc[:,1:7]
y_origin=df['Label']

In [52]:
train_x=X[:16047]
test_x=X[16047:]

train_y=y_origin[:16047]
test_y=y_origin[16047:]


smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_resample(train_x,train_y)

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', train_x.shape, train_y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 전 레이블 값 분포: \n', pd.Series(train_y).value_counts())
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (16047, 6) (16047,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (24510, 6) (24510,)
SMOTE 적용 전 레이블 값 분포: 
 -1    8170
 1    7731
 0     146
Name: Label, dtype: int64
SMOTE 적용 후 레이블 값 분포: 
  0    8170
 1    8170
-1    8170
Name: Label, dtype: int64


In [53]:
ms=MinMaxScaler()
X_train_over_ms=ms.fit_transform(X_train_over)
X_test_ms=ms.fit_transform(test_x)

y_train_over_dummies=pd.get_dummies(y_train_over)
test_y_dummies=pd.get_dummies(test_y)

train_x = X_train_over.values
test_x = X_test_ms
train_y = y_train_over_dummies.values
test_y=test_y_dummies.values

In [54]:
train_x_torch=torch.FloatTensor(train_x)
test_x_torch=torch.FloatTensor(test_x)

train_y_torch=torch.IntTensor(train_y)
test_y_torch=torch.IntTensor(test_y)

In [62]:
train_y_torch

tensor([[0, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        ...,
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]], dtype=torch.int32)

In [65]:
# 데이터 로더 준비
batch_size = 16

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

EPOCHS = 30
BATCH_SIZE = 71


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 3)

    def forward(self, x):
        x = x.view(-1, 6)  # 1차원 행렬로 변환
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)
epoches=300

In [66]:
for epoche in range(epoches+1):
    pred=model(train_x_torch)

    loss=F.nll_loss(pred, train_y_torch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
# def train(model, train_loader, optimizer):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(DEVICE), target.to(DEVICE)
#         optimizer.zero_grad()  # 기울기 초기화
#         output = model(data)
#         loss = F.cross_entropy(output, target)
#         loss.backward()
#         optimizer.step()


# def evaluate(model, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(DEVICE), target.to(DEVICE)
#             output = model(data)

#             test_loss += F.cross_entropy(output, target, reduction='sum').item()  # 오차 누적값

#             pred = output.max(1, keepdim=True)[1]  # output의 가장 큰 값의 인덱스 (argmax)
#             correct += pred.eq(target.view_as(pred)).sum().item()  # target, pred 일치하는 값의 개수

#         test_loss /= len(test_loader.dataset)  # 오차의 평균값
#         test_accuracy = 100 * correct / len(test_loader.dataset)
#         return test_loss, test_accuracy


# for epoch in range(1, EPOCHS + 1):
#     train(model, train_loader, optimizer)
#     test_loss, test_accuracy = evaluate(model, test_loader)

#     print(f'[{epoch}] Test Loss: {test_loss:.4f}, Accuracy:{test_accuracy:.2f}')